In [ ]:
import cv2
import json
import torch
import numpy as np
from datetime import datetime
from ultralytics import YOLO
from shapely.geometry import Point, Polygon
from collections import defaultdict
from sqlalchemy.orm import Session
from be_fastapi.app.database.session import SessionLocal
from be_fastapi.app.models.zone import Zone 

class VideoProcessor:
    def __init__(self, model_path='be_fastapi\\app\\engine\\ckpt\\best.onnx', source_id=None):
        """
        model_path: checkpoint (.pt or .onnx)
        source_id: source ID
        """
        self.source_id = source_id
        if model_path.endswith('.onnx'):
            self.device = 'cpu'
            print(f"[Source {self.source_id}] Running ONNX mode on CPU")
        else:
            self.device = 0 if torch.cuda.is_available() else 'cpu'
            print(f"[Source {self.source_id}] Using device: {torch.cuda.get_device_name(0) if self.device == 0 else 'CPU'}")

        self.model = YOLO(model_path, task='detect')

        self.CLASS_NAMES = {
            0: 'Car', 1: 'Bus', 2: 'Truck', 3: 'Motorcycle',
            4: 'Person', 5: 'Traffic Light', 
            6: 'Helmet', 7: 'No Helmet', 8: 'License Plate'
        }
        
        self.VEHICLE_CLASSES = [0, 1, 2, 3] 
        self.ATTRIBUTE_CLASSES = [6, 7, 8]
        
        # load zones from DB
        self.zones = {} 
        if self.source_id:
            self.load_zones_from_db(self.source_id)
        else:
            print("No source_id provided, cannot load zones.")
        
        # Temporary memory
        self.track_history = defaultdict(lambda: [])
        self.zone_entry_times = defaultdict(lambda: {})

    def load_zones_from_db(self, source_id):
        """
        Load zones from the database for the given source_id.
        """
        db: Session = SessionLocal()
        try:
            db_zones = db.query(Zone).filter(Zone.source_id == source_id).all()
            
            self.zones = {}
            for z in db_zones:
                try:
                    points = z.coordinates
                    if isinstance(points, str):
                        points = json.loads(points) #convert to list if stored as string
                    if points and isinstance(points, list) and len(points) >= 3:
                        poly = Polygon(points)
                        self.zones[z.name] = poly
                    else:
                        print(f"Zone '{z.name}' has invalid coordinates.")
                except Exception as e:
                    print(f"Error parsing zone '{z.name}': {e}")
        except Exception as e:
            print(f"Error connecting to Database when loading zones: {e}")
        finally:
            db.close()

    def calculate_iou(self, box1, box2):
        """
            Calculate Intersection over Union (IoU) of two bounding boxes.
            box1, box2: [x_min, y_min, x_max, y_max]
        """
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        intersection = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        
        if area1 + area2 - intersection == 0: return 0
        return intersection / (area1 + area2 - intersection)

    def estimate_speed(self, track_id, current_center):
        """
            Estimate speed in km/h based on pixel movement.
            track_id: ID of the tracked object
            sclae_factor: conversion factor from pixels to meters, in this case 0.05 m/pixel
        """
        scale_factor = 0.05
        if track_id not in self.track_history or len(self.track_history[track_id]) < 2:
            return 0.0
        prev_center = self.track_history[track_id][-1]
        pixel_dist = np.sqrt((current_center[0] - prev_center[0])**2 + (current_center[1] - prev_center[1])**2)
        speed_mps = pixel_dist * scale_factor * 30 
        return round(speed_mps * 3.6, 1)

    def process_video(self, video_source=0):
        cap = cv2.VideoCapture(video_source)
        
        while cap.isOpened():
            success, frame = cap.read()
            if not success: break

            # Run detection and tracking
            results = self.model.track(
                frame, 
                persist=True, 
                tracker="bytetrack.yaml", 
                device=self.device,
                verbose=False,
                conf=0.25
            )
            
            # If no detections, just show original frame
            if not results or not results[0].boxes:
                cv2.imshow(f"Cam {self.source_id}", frame)
                if cv2.waitKey(1) & 0xFF == ord("q"): break
                continue

            # Process results
            boxes = results[0].boxes
            current_timestamp = datetime.utcnow().isoformat() + "Z" # ISO 8601 format
            
            # Classify boxes into vehicles and attributes
            vehicles = []
            attributes = []
            for box in boxes:
                cls_id = int(box.cls[0])
                if cls_id in self.VEHICLE_CLASSES:
                    vehicles.append(box)
                elif cls_id in self.ATTRIBUTE_CLASSES:
                    attributes.append(box)

            # Process each vehicle
            dsl_objects = []
            for veh in vehicles:
                if veh.id is None: continue 
                
                track_id = int(veh.id[0])
                cls_id = int(veh.cls[0])
                x1, y1, x2, y2 = veh.xyxy[0].tolist()
                center_point = Point((x1 + x2) / 2, (y1 + y2) / 2)
                center_tuple = ((x1 + x2) / 2, (y1 + y2) / 2)

                self.track_history[track_id].append(center_tuple)
                if len(self.track_history[track_id]) > 30:
                    self.track_history[track_id].pop(0)

                # Ghép thuộc tính
                has_helmet = None
                license_text = "Unknown"
                veh_box = [x1, y1, x2, y2]
                
                for attr in attributes:
                    attr_box = attr.xyxy[0].tolist()
                    attr_id = int(attr.cls[0])
                    if self.calculate_iou(veh_box, attr_box) > 0.01:
                        if attr_id == 6: has_helmet = True
                        elif attr_id == 7: has_helmet = False
                        elif attr_id == 8: license_text = "DETECTED_PLATE"

                current_zone_name = None
                zone_durations = {}
                for z_name, z_poly in self.zones.items():
                    if z_poly.contains(center_point):
                        current_zone_name = z_name 
                        
                        if track_id not in self.zone_entry_times: self.zone_entry_times[track_id] = {}
                        if z_name not in self.zone_entry_times[track_id]: self.zone_entry_times[track_id][z_name] = datetime.now()
                        
                        duration = (datetime.now() - self.zone_entry_times[track_id][z_name]).total_seconds()
                        zone_durations[z_name] = round(duration, 1)
                    else:
                        if track_id in self.zone_entry_times and z_name in self.zone_entry_times[track_id]:
                            del self.zone_entry_times[track_id][z_name]

                obj_data = {
                    "track_id": track_id,
                    "class_name": self.CLASS_NAMES[cls_id],
                    "class_id": cls_id,
                    "bbox": [int(x1), int(y1), int(x2), int(y2)],
                    "confidence": float(veh.conf[0]),
                    "speed_kmh": self.estimate_speed(track_id, center_tuple),
                    "direction_angle": 0.0,
                    "current_zone": current_zone_name,
                    "zone_duration_seconds": zone_durations,
                    "attributes": {
                        "has_helmet": has_helmet,
                        "license_plate_text": license_text
                    }
                }
                dsl_objects.append(obj_data)

            final_json = {
                "source_id": str(self.source_id), # Gửi ID camera đi kèm
                "frame_timestamp": current_timestamp,
                "objects": dsl_objects
            }
            

            annotated_frame = results[0].plot()
            # Vẽ vùng lên hình để debug
            for z_name, z_poly in self.zones.items():
                pts = np.array(z_poly.exterior.coords, np.int32)
                pts = pts.reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [pts], True, (0, 255, 255), 2)
                cv2.putText(annotated_frame, z_name, (pts[0][0][0], pts[0][0][1]), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

            cv2.imshow(f"Cam {self.source_id}", annotated_frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    """
    Hàm main để test AI Engine độc lập.
    Chạy file này trực tiếp để debug và kiểm tra xem model có hoạt động đúng không.
    """
    print("="*60)
    print("🚀 Starting AI Engine Test Mode")
    print("="*60)
    
    # --- Cấu hình ---
    TEST_SOURCE_ID = 1  # ID của camera trong Database
    MODEL_PATH = 'be_fastapi\\app\\engine\\ckpt\\best.onnx'  # Đường dẫn model
    VIDEO_SOURCE = 'test_video.mp4'  # Có thể là file video hoặc 0 cho webcam
    
    # --- Khởi tạo Processor ---
    try:
        processor = VideoProcessor(
            model_path=MODEL_PATH,
            source_id=TEST_SOURCE_ID
        )
        
        print(f"✅ Model loaded successfully: {MODEL_PATH}")
        print(f"✅ Loaded {len(processor.zones)} zones from database for Source ID {TEST_SOURCE_ID}")
        
        if len(processor.zones) == 0:
            print("⚠️  WARNING: No zones found! Make sure database has zones for this source_id.")
        
        print("\n📹 Starting video processing...")
        print("   Press 'q' to quit\n")
        
        # --- Chạy xử lý video ---
        processor.process_video(video_source=VIDEO_SOURCE)
        
    except FileNotFoundError:
        print(f"❌ ERROR: Model file not found at: {MODEL_PATH}")
        print("   Please check the path and make sure the model file exists.")
    except Exception as e:
        print(f"❌ ERROR: {e}")
        import traceback
        traceback.print_exc()
    finally:
        print("\n" + "="*60)
        print("🛑 AI Engine stopped")
        print("="*60)

In [1]:
"""
Script test detection để xem model đang detect được gì
Chạy cell này để xem kết quả detection trực tiếp
"""
import cv2
import torch
from ultralytics import YOLO
from collections import defaultdict
import numpy as np

# ============ CẤU HÌNH ============
MODEL_PATH = '../../../app/engine/ckpt/best.pt'  # hoặc best.onnx
VIDEO_PATH = '../../../../videos/no_helmet.mp4'
CONF_THRESHOLD = 0.25
SHOW_EVERY_N_FRAMES = 1  # Hiển thị mỗi N frame (1 = tất cả)
MAX_FRAMES = 500  # Số frame tối đa để test (None = toàn bộ video)

# ============ CLASS NAMES ============
CLASS_NAMES = {
    0: 'Car', 1: 'Bus', 2: 'Truck', 3: 'Motorcycle',
    4: 'Person', 5: 'Traffic Light', 
    6: 'Helmet', 7: 'No Helmet', 8: 'License Plate'
}

VEHICLE_CLASSES = [0, 1, 2, 3]
ATTRIBUTE_CLASSES = [6, 7, 8]

# ============ LOAD MODEL ============
device = 0 if torch.cuda.is_available() else 'cpu'
print(f"🖥️  Device: {'GPU - ' + torch.cuda.get_device_name(0) if device == 0 else 'CPU'}")

model = YOLO(MODEL_PATH, task='detect')
print(f"✅ Model loaded: {MODEL_PATH}")

# ============ OPEN VIDEO ============
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise Exception(f"❌ Cannot open video: {VIDEO_PATH}")

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"📹 Video: {VIDEO_PATH}")
print(f"   Frames: {total_frames}, FPS: {fps}, Size: {width}x{height}")

# ============ DETECTION STATS ============
detection_counts = defaultdict(int)
frames_with_detections = 0

# ============ PROCESS VIDEO ============
print("\n🔍 Starting detection...")
print("=" * 60)

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1
    if MAX_FRAMES and frame_count > MAX_FRAMES:
        break
    
    # Run detection
    results = model(frame, device=device, verbose=False, conf=CONF_THRESHOLD)
    
    if results and results[0].boxes and len(results[0].boxes) > 0:
        boxes = results[0].boxes
        frames_with_detections += 1
        
        # Count detections
        frame_detections = defaultdict(int)
        for box in boxes:
            cls_id = int(box.cls[0])
            cls_name = CLASS_NAMES.get(cls_id, f"Unknown({cls_id})")
            detection_counts[cls_name] += 1
            frame_detections[cls_name] += 1
        
        # Print frame info (mỗi 50 frame hoặc khi có detection đặc biệt)
        if frame_count % 50 == 0 or 'No Helmet' in frame_detections or 'Motorcycle' in frame_detections:
            print(f"Frame {frame_count:4d}/{total_frames}: {dict(frame_detections)}")

cap.release()

# ============ SUMMARY ============
print("=" * 60)
print("\n📊 DETECTION SUMMARY")
print("=" * 60)
print(f"Total frames processed: {frame_count}")
print(f"Frames with detections: {frames_with_detections} ({frames_with_detections/frame_count*100:.1f}%)")
print(f"\nDetection counts by class:")
print("-" * 40)

for cls_name, count in sorted(detection_counts.items(), key=lambda x: -x[1]):
    avg_per_frame = count / frame_count
    icon = "🏍️" if cls_name == "Motorcycle" else "🚗" if cls_name == "Car" else "⛑️" if cls_name == "Helmet" else "❌" if cls_name == "No Helmet" else "📋" if cls_name == "License Plate" else "📦"
    print(f"  {icon} {cls_name:15s}: {count:6d} (avg {avg_per_frame:.2f}/frame)")

print("-" * 40)

# ============ HIGHLIGHT KEY FINDINGS ============
print("\n🎯 KEY FINDINGS:")
if 'Motorcycle' in detection_counts:
    print(f"   ✅ Motorcycles detected: {detection_counts['Motorcycle']}")
else:
    print(f"   ⚠️  No motorcycles detected!")

if 'No Helmet' in detection_counts:
    print(f"   🚨 NO HELMET violations: {detection_counts['No Helmet']}")
else:
    print(f"   ℹ️  No 'No Helmet' detections")

if 'Helmet' in detection_counts:
    print(f"   ⛑️  Helmets detected: {detection_counts['Helmet']}")

if 'License Plate' in detection_counts:
    print(f"   📋 License plates detected: {detection_counts['License Plate']}")

print("\n✅ Detection test completed!")

🖥️  Device: GPU - NVIDIA GeForce RTX 4060 Laptop GPU
✅ Model loaded: ../../../app/engine/ckpt/best.pt
📹 Video: ../../../../videos/no_helmet.mp4
   Frames: 598, FPS: 25.0, Size: 1920x1080

🔍 Starting detection...
Frame   39/598: {'Helmet': 4, 'No Helmet': 3}
Frame   40/598: {'Helmet': 3, 'No Helmet': 3}
Frame   41/598: {'Helmet': 4, 'No Helmet': 4}
Frame   42/598: {'Helmet': 5, 'No Helmet': 3}
Frame   43/598: {'Helmet': 5, 'No Helmet': 2}
Frame   44/598: {'Helmet': 4, 'No Helmet': 3}
Frame   45/598: {'Helmet': 4, 'No Helmet': 4}
Frame   46/598: {'Helmet': 5, 'No Helmet': 4}
Frame   47/598: {'Helmet': 4, 'No Helmet': 6}
Frame   48/598: {'Helmet': 4, 'No Helmet': 4}
Frame   49/598: {'Helmet': 4, 'No Helmet': 5}
Frame   50/598: {'Helmet': 2, 'No Helmet': 8}
Frame   51/598: {'Helmet': 2, 'No Helmet': 6}
Frame   52/598: {'Helmet': 3, 'No Helmet': 7}
Frame   53/598: {'Helmet': 5, 'No Helmet': 6}
Frame   54/598: {'Helmet': 5, 'No Helmet': 7}
Frame   55/598: {'Helmet': 4, 'No Helmet': 5}
Frame 

In [2]:
"""
🎬 REALTIME VISUALIZATION
Hiển thị video với bounding boxes và thông tin detection
"""
import cv2
import torch
from ultralytics import YOLO
from collections import defaultdict
import numpy as np
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from PIL import Image

# ============ CẤU HÌNH ============
MODEL_PATH = '../../../app/engine/ckpt/best.pt'
VIDEO_PATH = '../../../../videos/dau_vung_cam.mp4'
CONF_THRESHOLD = 0.25
DISPLAY_SIZE = (960, 540)  # Resize để hiển thị nhanh hơn

# ============ CLASS CONFIG ============
CLASS_NAMES = {
    0: 'Car', 1: 'Bus', 2: 'Truck', 3: 'Motorcycle',
    4: 'Person', 5: 'Traffic Light', 
    6: 'Helmet', 7: 'No Helmet', 8: 'License Plate'
}

# Màu sắc cho từng class (BGR)
CLASS_COLORS = {
    0: (0, 255, 0),      # Car - Green
    1: (255, 165, 0),    # Bus - Orange
    2: (0, 0, 255),      # Truck - Red
    3: (255, 255, 0),    # Motorcycle - Cyan
    4: (128, 0, 128),    # Person - Purple
    5: (0, 255, 255),    # Traffic Light - Yellow
    6: (0, 255, 0),      # Helmet - Green
    7: (0, 0, 255),      # No Helmet - RED (quan trọng!)
    8: (255, 0, 255),    # License Plate - Magenta
}

# ============ LOAD MODEL ============
device = 0 if torch.cuda.is_available() else 'cpu'
print(f"🖥️  Device: {'GPU - ' + torch.cuda.get_device_name(0) if device == 0 else 'CPU'}")

model = YOLO(MODEL_PATH, task='detect')
print(f"✅ Model loaded: {MODEL_PATH}")

# ============ OPEN VIDEO ============
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise Exception(f"❌ Cannot open video: {VIDEO_PATH}")

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"📹 Video: {total_frames} frames @ {fps} FPS")

# ============ VISUALIZATION FUNCTION ============
def draw_detections(frame, boxes, class_names, class_colors, conf_threshold=0.25):
    """Vẽ bounding boxes và labels lên frame"""
    detection_info = defaultdict(int)
    
    for box in boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        
        if conf < conf_threshold:
            continue
            
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        cls_name = class_names.get(cls_id, f"Unknown({cls_id})")
        color = class_colors.get(cls_id, (128, 128, 128))
        
        detection_info[cls_name] += 1
        
        # Vẽ box - dày hơn cho No Helmet
        thickness = 3 if cls_id == 7 else 2
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
        
        # Label background
        label = f"{cls_name} {conf:.2f}"
        (label_w, label_h), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(frame, (x1, y1 - label_h - 10), (x1 + label_w, y1), color, -1)
        cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        # Thêm icon cho No Helmet
        if cls_id == 7:  # No Helmet
            cv2.putText(frame, "⚠️ VIOLATION!", (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    return frame, detection_info

def draw_stats_overlay(frame, detection_info, frame_num, total_frames):
    """Vẽ overlay thống kê lên góc màn hình"""
    overlay = frame.copy()
    h, w = frame.shape[:2]
    
    # Background panel
    cv2.rectangle(overlay, (10, 10), (280, 150), (0, 0, 0), -1)
    cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
    
    # Title
    cv2.putText(frame, f"Frame: {frame_num}/{total_frames}", (20, 35), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # Detection counts
    y_offset = 60
    for cls_name, count in sorted(detection_info.items()):
        color = (0, 255, 0) if cls_name not in ['No Helmet'] else (0, 0, 255)
        cv2.putText(frame, f"{cls_name}: {count}", (20, y_offset), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        y_offset += 20
    
    return frame

# ============ REALTIME DISPLAY ============
print("\n🎬 Starting realtime visualization...")
print("   Displaying frames in notebook (every 3rd frame for speed)")
print("-" * 50)

fig, ax = plt.subplots(1, 1, figsize=(14, 8))
plt.ion()  # Interactive mode

frame_count = 0
skip_frames = 2  # Hiển thị mỗi 3 frame để mượt hơn trong notebook

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # Skip frames để tăng tốc
        if frame_count % (skip_frames + 1) != 0:
            continue
        
        # Run detection
        results = model(frame, device=device, verbose=False, conf=CONF_THRESHOLD)
        
        # Draw detections
        detection_info = {}
        if results and results[0].boxes and len(results[0].boxes) > 0:
            frame, detection_info = draw_detections(
                frame, results[0].boxes, CLASS_NAMES, CLASS_COLORS, CONF_THRESHOLD
            )
        
        # Draw stats overlay
        frame = draw_stats_overlay(frame, detection_info, frame_count, total_frames)
        
        # Resize for display
        frame_resized = cv2.resize(frame, DISPLAY_SIZE)
        
        # Convert BGR to RGB for matplotlib
        frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
        
        # Display in notebook
        clear_output(wait=True)
        ax.clear()
        ax.imshow(frame_rgb)
        ax.axis('off')
        ax.set_title(f"🎬 Realtime Detection - Frame {frame_count}/{total_frames}", fontsize=14)
        
        # Print current detections
        if detection_info:
            det_str = ", ".join([f"{k}: {v}" for k, v in detection_info.items()])
            print(f"Frame {frame_count}: {det_str}")
            if 'No Helmet' in detection_info:
                print(f"   🚨 VIOLATION!!!!")
        
        display(fig)
        
        # Dừng sớm nếu cần (nhấn interrupt kernel)
        if frame_count > 1000:  # Giới hạn 300 frames để test
            print("\n⏹️ Stopped at 300 frames for demo")
            break

except KeyboardInterrupt:
    print("\n⏹️ Stopped by user")
finally:
    cap.release()
    plt.close(fig)
    print(f"\n✅ Processed {frame_count} frames")
    print("🎬 Visualization completed!")

Frame 507: Car: 6

⏹️ Stopped by user

✅ Processed 507 frames
🎬 Visualization completed!
